The reference of this notebook is as follows :

1.https://tensorflow.google.cn/tutorials/text/transformer

2.our host's papers (https://arxiv.org/abs/2002.07033, https://arxiv.org/abs/2010.12042)

![](http://)

# **Main ideas is as follows：**

1.Basic transformer：  Portugese-English translation . I consider contend_id as Portugese，answered_correctly as English, so SAINT is contend_id -answered_correctly translation.


2.modify create_masks function


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
from sklearn.metrics import roc_auc_score
import time
import matplotlib.pyplot as plt

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os


# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# **Data**

In [ ]:
target = 'answered_correctly'
content_count=13523
target_count=2

MAX_LENGTH=50

BUFFER_SIZE = 2000
BATCH_SIZE = 128

#train parameter
#The values used in the base model of transformer were:
#num_layers=6, d_model = 512, dff = 2048. See the paper for all the other versions of the transformer.

#SAINT:The window size, dropout rate, and batch size are set to 100, 0.1, and 128 respectively.
#The best performing model of SAINT has 4 layers and a latent space dimension of 512.
num_layers = 4
d_model = 128
dff = 512
num_heads = 8

# input_vocab_size = content_count + 1
# target_vocab_size = target_count + 1

#content_id have 0，so add 1
input_vocab_size = content_count+1
target_vocab_size = target_count
dropout_rate = 0.1

#
EPOCHS = 5

In [ ]:
%%time
dtype = {'timestamp':'int64', 
         'user_id':'int32' ,
         'content_id':'int16',
         'content_type_id':'int8',
         'answered_correctly':'int8'}

train_df = pd.read_csv('/kaggle/input/riiid-test-answer-prediction/train.csv', 
                       usecols=[1, 2, 3, 4, 7], 
                       dtype=dtype,
                       nrows=3300*10000
                      )

In [ ]:
##content_id have 0，so add 1
train_df['content_id'] += 1
# train_df['answered_correctly'] += 1


train_df = train_df[train_df.content_type_id == False]

valid_df=train_df[3000*10000:]
train_df=train_df[0:100*10000]

train_dataset = train_df[['user_id', 'content_id', 'answered_correctly']].groupby('user_id').apply(lambda r:[
            r['content_id'].values,
            r['answered_correctly'].values])

valid_dataset = valid_df[['user_id', 'content_id', 'answered_correctly']].groupby('user_id').apply(lambda r:[
            r['content_id'].values,
            r['answered_correctly'].values])

In [ ]:
del train_df
del valid_df

In [ ]:
print('len(train_dataset):',len(train_dataset))
print('len(valid_dataset):',len(valid_dataset))

In [ ]:
train_dataset=pd.DataFrame(train_dataset)
train_dataset=train_dataset.reset_index()
train_dataset.columns = ['user_id','q_qa']
#
valid_dataset=pd.DataFrame(valid_dataset)
valid_dataset=valid_dataset.reset_index()
valid_dataset.columns = ['user_id','q_qa']

In [ ]:
def getlimitdata(x):    
    seq_len = len(x)
    r = np.zeros(MAX_LENGTH, dtype=int)

    if seq_len >= MAX_LENGTH:          
        r[:] = x[-MAX_LENGTH:]
       
    else:        
        r[-seq_len:] = x
            
    return r
    

In [ ]:
train_dataset['q']=train_dataset['q_qa'].apply(lambda r:getlimitdata(r[0]))
train_dataset['qa']=train_dataset['q_qa'].apply(lambda r:getlimitdata(r[1]))
train_dataset.drop(columns=['q_qa'], inplace=True)

valid_dataset['q']=valid_dataset['q_qa'].apply(lambda r:getlimitdata(r[0]))
valid_dataset['qa']=valid_dataset['q_qa'].apply(lambda r:getlimitdata(r[1]))
valid_dataset.drop(columns=['q_qa'], inplace=True)

In [ ]:
train_dataset

In [ ]:
qs=[]
for q in train_dataset['q'].values:
    qs.append(q.tolist())
qas=[]
for qa in train_dataset['qa'].values:
    qas.append(qa.tolist())
    
qs_valid=[]
for q in valid_dataset['q'].values:
    qs_valid.append(q.tolist())
qas_valid=[]
for qa in valid_dataset['qa'].values:
    qas_valid.append(qa.tolist())

In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices((qs,qas))
valid_dataset = tf.data.Dataset.from_tensor_slices((qs_valid,qas_valid))

In [ ]:
del qs
del qas
del qs_valid
del qas_valid

In [ ]:
train_dataset = train_dataset.cache()
train_dataset = train_dataset.shuffle(BUFFER_SIZE).padded_batch(BATCH_SIZE)
train_dataset = train_dataset.prefetch(tf.data.experimental.AUTOTUNE)



In [ ]:
valid_dataset = valid_dataset.padded_batch(2000)#BATCH_SIZE

In [ ]:
q_batch, qa_batch =next(iter(train_dataset))
q_batch, qa_batch

In [ ]:
qa_batch

In [ ]:
qa_batch[:, :-1]

In [ ]:
#tar_real = 
qa_batch[:, 1:]

# **Model**

In [ ]:
def get_angles(pos, i, d_model):
    angle_rates = 1 / np.power(10000, (2 * (i//2)) / np.float32(d_model))
    return pos * angle_rates

def positional_encoding(position, d_model):
    angle_rads = get_angles(np.arange(position)[:, np.newaxis],
                          np.arange(d_model)[np.newaxis, :],
                          d_model)

    # apply sin to even indices in the array; 2i
    angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])

    # apply cos to odd indices in the array; 2i+1
    angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])

    pos_encoding = angle_rads[np.newaxis, ...]

    return tf.cast(pos_encoding, dtype=tf.float32)

# **Mask**

In [ ]:
def create_padding_mask(seq):
    seq = tf.cast(tf.math.equal(seq, 0), tf.float32)#

    # add extra dimensions to add the padding
    # to the attention logits.
    return seq[:, tf.newaxis, tf.newaxis, :]  # (batch_size, 1, 1, seq_len)

def create_look_ahead_mask(size):
    mask = 1 - tf.linalg.band_part(tf.ones((size, size)), -1, 0)
    return mask  # (seq_len, seq_len)

# **Scaled dot product attention**

In [ ]:
def scaled_dot_product_attention(q, k, v, mask):
#     """Calculate the attention weights.
#       q, k, v must have matching leading dimensions.
#       k, v must have matching penultimate dimension, i.e.: seq_len_k = seq_len_v.
#       The mask has different shapes depending on its type(padding or look ahead) 
#       but it must be broadcastable for addition.

#       Args:
#         q: query shape == (..., seq_len_q, depth)
#         k: key shape == (..., seq_len_k, depth)
#         v: value shape == (..., seq_len_v, depth_v)
#         mask: Float tensor with shape broadcastable 
#               to (..., seq_len_q, seq_len_k). Defaults to None.

#       Returns:
#         output, attention_weights
#     """

    matmul_qk = tf.matmul(q, k, transpose_b=True)  # (..., seq_len_q, seq_len_k)

    # scale matmul_qk
    dk = tf.cast(tf.shape(k)[-1], tf.float32)
    scaled_attention_logits = matmul_qk / tf.math.sqrt(dk)

    # add the mask to the scaled tensor.
    if mask is not None:
        scaled_attention_logits += (mask * -1e9)  

    # softmax is normalized on the last axis (seq_len_k) so that the scores
    # add up to 1.
    attention_weights = tf.nn.softmax(scaled_attention_logits, axis=-1)  # (..., seq_len_q, seq_len_k)

    output = tf.matmul(attention_weights, v)  # (..., seq_len_q, depth_v)

    return output, attention_weights

# **Multi-head attention**

In [ ]:
class MultiHeadAttention(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        self.num_heads = num_heads
        self.d_model = d_model

        assert d_model % self.num_heads == 0

        self.depth = d_model // self.num_heads

        self.wq = tf.keras.layers.Dense(d_model)
        self.wk = tf.keras.layers.Dense(d_model)
        self.wv = tf.keras.layers.Dense(d_model)

        self.dense = tf.keras.layers.Dense(d_model)

    def split_heads(self, x, batch_size):
       
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.depth))
        return tf.transpose(x, perm=[0, 2, 1, 3])

    def call(self, v, k, q, mask):
        batch_size = tf.shape(q)[0]

        q = self.wq(q)  # (batch_size, seq_len, d_model)
        k = self.wk(k)  # (batch_size, seq_len, d_model)
        v = self.wv(v)  # (batch_size, seq_len, d_model)

        q = self.split_heads(q, batch_size)  # (batch_size, num_heads, seq_len_q, depth)
        k = self.split_heads(k, batch_size)  # (batch_size, num_heads, seq_len_k, depth)
        v = self.split_heads(v, batch_size)  # (batch_size, num_heads, seq_len_v, depth)

        # scaled_attention.shape == (batch_size, num_heads, seq_len_q, depth)
        # attention_weights.shape == (batch_size, num_heads, seq_len_q, seq_len_k)
        scaled_attention, attention_weights = scaled_dot_product_attention(
            q, k, v, mask)

        scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])  # (batch_size, seq_len_q, num_heads, depth)

        concat_attention = tf.reshape(scaled_attention, 
                                      (batch_size, -1, self.d_model))  # (batch_size, seq_len_q, d_model)

        output = self.dense(concat_attention)  # (batch_size, seq_len_q, d_model)

        return output, attention_weights

# **Point wise feed forward network**

In [ ]:
def point_wise_feed_forward_network(d_model, dff):
    return tf.keras.Sequential([
      tf.keras.layers.Dense(dff, activation='relu'),  # (batch_size, seq_len, dff)
      tf.keras.layers.Dense(d_model)  # (batch_size, seq_len, d_model)
    ])

# **Encoder and decoder**

In [ ]:
class EncoderLayer(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads, dff, rate=0.1):
        super(EncoderLayer, self).__init__()

        self.mha = MultiHeadAttention(d_model, num_heads)
        self.ffn = point_wise_feed_forward_network(d_model, dff)

        self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

        self.dropout1 = tf.keras.layers.Dropout(rate)
        self.dropout2 = tf.keras.layers.Dropout(rate)

    def call(self, x, training, mask):

        attn_output, _ = self.mha(x, x, x, mask)  # (batch_size, input_seq_len, d_model)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(x + attn_output)  # (batch_size, input_seq_len, d_model)

        ffn_output = self.ffn(out1)  # (batch_size, input_seq_len, d_model)
        ffn_output = self.dropout2(ffn_output, training=training)
        out2 = self.layernorm2(out1 + ffn_output)  # (batch_size, input_seq_len, d_model)

        return out2

In [ ]:
class Encoder(tf.keras.layers.Layer):
    def __init__(self, num_layers, d_model, num_heads, dff, input_vocab_size,
               maximum_position_encoding, rate=0.1):
        super(Encoder, self).__init__()

        self.d_model = d_model
        self.num_layers = num_layers

        self.embedding = tf.keras.layers.Embedding(input_vocab_size, d_model)
        self.pos_encoding = positional_encoding(maximum_position_encoding, 
                                                self.d_model)


        self.enc_layers = [EncoderLayer(d_model, num_heads, dff, rate) 
                           for _ in range(num_layers)]

        self.dropout = tf.keras.layers.Dropout(rate)

    def call(self, x, training, mask):

        seq_len = tf.shape(x)[1]

        # 
        x = self.embedding(x)  # (batch_size, input_seq_len, d_model)
        x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
        x += self.pos_encoding[:, :seq_len, :]

        x = self.dropout(x, training=training)

        for i in range(self.num_layers):
            x = self.enc_layers[i](x, training, mask)

        return x  # (batch_size, input_seq_len, d_model)

In [ ]:
class DecoderLayer(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads, dff, rate=0.1):
        super(DecoderLayer, self).__init__()

        self.mha1 = MultiHeadAttention(d_model, num_heads)
        self.mha2 = MultiHeadAttention(d_model, num_heads)

        self.ffn = point_wise_feed_forward_network(d_model, dff)

        self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm3 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

        self.dropout1 = tf.keras.layers.Dropout(rate)
        self.dropout2 = tf.keras.layers.Dropout(rate)
        self.dropout3 = tf.keras.layers.Dropout(rate)


    def call(self, x, enc_output, training, 
           look_ahead_mask, padding_mask):
    # enc_output.shape == (batch_size, input_seq_len, d_model)

        attn1, attn_weights_block1 = self.mha1(x, x, x, look_ahead_mask)  # (batch_size, target_seq_len, d_model)
        attn1 = self.dropout1(attn1, training=training)
        out1 = self.layernorm1(attn1 + x)

        attn2, attn_weights_block2 = self.mha2(
            enc_output, enc_output, out1, padding_mask)  # (batch_size, target_seq_len, d_model)
        attn2 = self.dropout2(attn2, training=training)
        out2 = self.layernorm2(attn2 + out1)  # (batch_size, target_seq_len, d_model)

        ffn_output = self.ffn(out2)  # (batch_size, target_seq_len, d_model)
        ffn_output = self.dropout3(ffn_output, training=training)
        out3 = self.layernorm3(ffn_output + out2)  # (batch_size, target_seq_len, d_model)

        return out3, attn_weights_block1, attn_weights_block2

In [ ]:
class Decoder(tf.keras.layers.Layer):
    def __init__(self, num_layers, d_model, num_heads, dff, target_vocab_size,
               maximum_position_encoding, rate=0.1):
        super(Decoder, self).__init__()

        self.d_model = d_model
        self.num_layers = num_layers

        self.embedding = tf.keras.layers.Embedding(target_vocab_size, d_model)
        self.pos_encoding = positional_encoding(maximum_position_encoding, d_model)

        self.dec_layers = [DecoderLayer(d_model, num_heads, dff, rate) 
                           for _ in range(num_layers)]
        self.dropout = tf.keras.layers.Dropout(rate)

    def call(self, x, enc_output, training, 
           look_ahead_mask, padding_mask):

        seq_len = tf.shape(x)[1]
        attention_weights = {}

        x = self.embedding(x)  # (batch_size, target_seq_len, d_model)
        x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
        x += self.pos_encoding[:, :seq_len, :]

        x = self.dropout(x, training=training)

        for i in range(self.num_layers):
            x, block1, block2 = self.dec_layers[i](x, enc_output, training,
                                                 look_ahead_mask, padding_mask)

            attention_weights['decoder_layer{}_block1'.format(i+1)] = block1
            attention_weights['decoder_layer{}_block2'.format(i+1)] = block2

        # x.shape == (batch_size, target_seq_len, d_model)
        return x, attention_weights

# ** Transformer**

In [ ]:
class Transformer(tf.keras.Model):
    def __init__(self, num_layers, d_model, num_heads, dff, input_vocab_size, 
               target_vocab_size, pe_input, pe_target, rate=0.1):
        super(Transformer, self).__init__()

        self.encoder = Encoder(num_layers, d_model, num_heads, dff, 
                               input_vocab_size, pe_input, rate)

        self.decoder = Decoder(num_layers, d_model, num_heads, dff, 
                               target_vocab_size, pe_target, rate)

        self.final_layer = tf.keras.layers.Dense(target_vocab_size,activation='softmax')

    def call(self, inp, tar, training, enc_padding_mask, 
           look_ahead_mask, dec_padding_mask):

        enc_output = self.encoder(inp, training, enc_padding_mask)  # (batch_size, inp_seq_len, d_model)

        # dec_output.shape == (batch_size, tar_seq_len, d_model)
        dec_output, attention_weights = self.decoder(
            tar, enc_output, training, look_ahead_mask, dec_padding_mask)

        final_output = self.final_layer(dec_output)  # (batch_size, tar_seq_len, target_vocab_size)
        
        #增加的        
        #final_output = final_output[: ,-1:, :]  # (batch_size, 1, vocab_size)
#         final_output = tf.keras.layers.GlobalAveragePooling1D()(dec_output)
#         final_output = tf.keras.layers.Dropout(0.1)(final_output)         
#         final_output = tf.keras.layers.Dense(1, activation='sigmoid', name='output')(final_output)

        return final_output, attention_weights

# **Optimizer**

In [ ]:
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, d_model, warmup_steps=4000):
        super(CustomSchedule, self).__init__()

        self.d_model = d_model
        self.d_model = tf.cast(self.d_model, tf.float32)

        self.warmup_steps = warmup_steps

    def __call__(self, step):
        arg1 = tf.math.rsqrt(step)
        arg2 = step * (self.warmup_steps ** -1.5)

        return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)

In [ ]:
learning_rate = CustomSchedule(d_model)
#SAINT:We use the Adam optimizer  with lr =0:001;b1 = 0:9;b2 = 0:999 and epsilon = 1e􀀀8.
optimizer = tf.keras.optimizers.Adam(learning_rate, beta_1=0.9, beta_2=0.98, 
                                     epsilon=1e-9)

# **Loss and metrics**

In [ ]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

def loss_function(mask,real, pred):
    #mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_ = loss_object(real, pred)

    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask

    return tf.reduce_sum(loss_)/tf.reduce_sum(mask)


def accuracy_function(mask,real, pred):
    real = tf.cast(real, dtype=tf.int64)
    accuracies = tf.equal(real, tf.argmax(pred, axis=2))

    #mask = tf.math.logical_not(tf.math.equal(real, 0))
    accuracies = tf.math.logical_and(mask, accuracies)

    accuracies = tf.cast(accuracies, dtype=tf.float32)
    mask = tf.cast(mask, dtype=tf.float32)
    return tf.reduce_sum(accuracies)/tf.reduce_sum(mask)

train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.Mean(name='train_accuracy')

# **Training and checkpointing**

In [ ]:
transformer = Transformer(num_layers, d_model, num_heads, dff,
                          input_vocab_size, target_vocab_size, 
                          pe_input=50, 
                          pe_target=50,
                          rate=dropout_rate)

In [ ]:
def create_masks(inp, tar):
    # 
    enc_padding_mask = create_look_ahead_mask(tf.shape(inp)[1]) 
    dec_padding_mask = create_look_ahead_mask(tf.shape(inp)[1])
    dec_padding_mask=dec_padding_mask[1:,]
    combined_mask = create_look_ahead_mask(tf.shape(tar)[1])

    return enc_padding_mask, combined_mask, dec_padding_mask

In [ ]:
# def create_masks(inp, tar):
#     # 
#     enc_padding_mask = create_padding_mask(inp)
#     look_ahead_mask_in = create_look_ahead_mask(tf.shape(inp)[1])
#     enc_padding_mask = tf.maximum(enc_padding_mask, look_ahead_mask_in)
    
#     # Used in the 2nd attention block in the decoder.
#     # This padding mask is used to mask the encoder outputs.
#     dec_padding_mask = create_padding_mask(inp)

#     # Used in the 1st attention block in the decoder.
#     # It is used to pad and mask future tokens in the input received by 
#     # the decoder.
#     look_ahead_mask = create_look_ahead_mask(tf.shape(tar)[1])
    
#     #tar_inp = tar[:, :-1] input is same with out,but input have no 0,so use input create mask
#     #dec_target_padding_mask = create_padding_mask(tar)
#     dec_target_padding_mask = create_padding_mask(inp[:, :-1])
#     combined_mask = tf.maximum(dec_target_padding_mask, look_ahead_mask)

#     return enc_padding_mask, combined_mask, dec_padding_mask

In [ ]:
checkpoint_path = "../input/riiid-saint-tf-transformer/checkpoints/train"

ckpt = tf.train.Checkpoint(transformer=transformer,
                           optimizer=optimizer)

ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=5)

# 
if ckpt_manager.latest_checkpoint:
    ckpt.restore(ckpt_manager.latest_checkpoint)
    print ('Latest checkpoint restored!!')

In [ ]:
# The @tf.function trace-compiles train_step into a TF graph for faster
# execution. The function specializes to the precise shape of the argument
# tensors. To avoid re-tracing due to the variable sequence lengths or variable
# batch sizes (the last batch is smaller), use input_signature to specify
# more generic shapes.

# train_step_signature = [
#     tf.TensorSpec(shape=(None, None), dtype=tf.int64),
#     tf.TensorSpec(shape=(None, None), dtype=tf.int64),
# ]

# @tf.function(input_signature=train_step_signature)
def train_step(inp, tar):
    tar_inp = tar[:, :-1]
    tar_real = tar[:, 1:]
    #sigmoid,just last tar
    #tar_real=tar[: ,-1]
    
    mask = tf.math.logical_not(tf.math.equal(inp[:, 1:], 0))

    enc_padding_mask, combined_mask, dec_padding_mask = create_masks(inp, tar_inp)

    with tf.GradientTape() as tape:
        predictions, _ = transformer(inp, tar_inp, 
                                     True, 
                                     enc_padding_mask, 
                                     combined_mask, 
                                     dec_padding_mask)
        loss = loss_function(mask,tar_real, predictions)

    gradients = tape.gradient(loss, transformer.trainable_variables)    
    optimizer.apply_gradients(zip(gradients, transformer.trainable_variables))

    train_loss(loss)
    train_accuracy(accuracy_function(mask,tar_real, predictions))

In [ ]:
print('start train....')

In [ ]:
# for epoch in range(EPOCHS):
#     start = time.time()

#     train_loss.reset_states()
#     train_accuracy.reset_states()

#     # inp -> portuguese, tar -> english
#     for (batch, (inp, tar)) in enumerate(train_dataset):
#         train_step(inp, tar)

#         if batch % 10 == 0:
#             print ('Epoch {} Batch {} Loss {:.4f} accuracy {:.4f}'.format(
#               epoch + 1, batch, train_loss.result(), train_accuracy.result()))

#     if (epoch + 1) % 5 == 0:
#         ckpt_save_path = ckpt_manager.save()
#         print ('Saving checkpoint for epoch {} at {}'.format(epoch+1,
#                                                              ckpt_save_path))

#     print ('Epoch {} Loss {:.4f} accuracy {:.4f}'.format(epoch + 1, 
#                                                 train_loss.result(), 
#                                                 train_accuracy.result()))

#     print ('Time taken for 1 epoch: {} secs\n'.format(time.time() - start))

In [ ]:
del train_dataset

In [ ]:
import datetime

In [ ]:
%%time
i=0
for (batch, (encoder_input, qa_batch)) in enumerate(valid_dataset):
    start = time.time()
    #label=qa_batch[:, 1:]
    label=qa_batch[: ,-1]
    tar_inp = qa_batch[:, :-1]
    print(encoder_input.shape)

    enc_padding_mask, combined_mask, dec_padding_mask = create_masks(
            encoder_input, tar_inp)
    
    predictions, attention_weights = transformer(encoder_input, 
                                                     tar_inp,
                                                     False,
                                                     enc_padding_mask,
                                                     combined_mask,
                                                     dec_padding_mask)
    
    predictions = predictions[: ,-1:, :]  # (batch_size, 1, vocab_size)
    predictions=tf.nn.softmax(predictions)[:,:,-1]
    end = time.time()
    print (end - start)
    print('valid-auc:', roc_auc_score(label, predictions))
    i=i+1
    if i==3:
        break
    

In [ ]:
print(encoder_input.shape)

In [ ]:
dtype = {'timestamp':'int64', 
         'user_id':'int32' ,
         'content_id':'int16',
         'content_type_id':'int8',
         'answered_correctly':'int8'}

train_df = pd.read_csv('/kaggle/input/riiid-test-answer-prediction/train.csv', 
                       usecols=[1, 2, 3, 4, 7], 
                       dtype=dtype,
                       #nrows=120*10000
                      )
train_df = train_df[train_df.content_type_id == False]
#when train ,content_id add 1
train_df['content_id'] += 1

In [ ]:

train_dataset = train_df[['user_id', 'content_id', 'answered_correctly']].groupby('user_id').apply(lambda r:[
            r['content_id'].values,
            r['answered_correctly'].values])

In [ ]:
del train_df

In [ ]:
train_dataset[115]

In [ ]:
from collections import defaultdict
train_dataset=train_dataset.to_dict(defaultdict(int))

In [ ]:
train_dataset[115]

In [ ]:
%%time
if 10000 in train_dataset.keys():
    print(100)

In [ ]:
%%time
train_dataset[110]

In [ ]:
%%time
#train_dataset[115][0]=np.concatenate((train_dataset[115][0],[100+1]))
#train_dataset2[115]
train_dataset[110]=[np.array([100+1]),np.array([1])]

In [ ]:
def getpredata(user_id,content_id):
    #
    if user_id in train_dataset.keys():
        #produce encoder_input, tar_inp
        q_qa=train_dataset[user_id]
        q=q_qa[0]
        qa=q_qa[1]
        #q.append(content_id)
        #concatenate new content_id
        q=np.concatenate((q,[content_id]))
        #
        q=getlimitdata(q)
        qa=getlimitdata(qa)
    else:
        q = np.zeros(MAX_LENGTH-1, dtype=int)
        q=np.concatenate((q,[content_id]))
        qa=np.zeros(MAX_LENGTH, dtype=int)
    
    return q, qa
    

In [ ]:
def getpredatas(user_ids,content_ids):
    encoder_inputs=[]
    tar_inps=[]
    for user_id, content_id in zip(user_ids,content_ids):
        encoder_input, tar_inp=getpredata(user_id,content_id)
        tar_inp=tar_inp[1:]#
        
        encoder_inputs.append(encoder_input.tolist())
        tar_inps.append(tar_inp.tolist())
    
    encoder_inputs=tf.convert_to_tensor(encoder_inputs)
    tar_inps=tf.convert_to_tensor(tar_inps)
    return encoder_inputs,tar_inps
        

In [ ]:
test_df = pd.read_csv('/kaggle/input/riiid-test-answer-prediction/train.csv', 
                       usecols=[1, 2, 3, 4, 7], 
                       dtype=dtype,
                       nrows=1*10000
                      )
test_df=test_df[0:108]

In [ ]:
%%time
start = time.time()
test_df = test_df[test_df['content_type_id'] == 0].reset_index(drop=True)
print(len(test_df))
user_ids = test_df['user_id'].values
content_ids = test_df['content_id'].values


encoder_inputs, tar_inps=getpredatas(user_ids,content_ids)

enc_padding_mask, combined_mask, dec_padding_mask = create_masks(
    encoder_inputs, tar_inps)
print(time.time()-start)
predictions, attention_weights = transformer(encoder_inputs, 
                                             tar_inps,
                                             False,
                                             enc_padding_mask,
                                             combined_mask,
                                             dec_padding_mask)
print(time.time()-start)
predictions = predictions[: ,-1:, :]  # (batch_size, 1, vocab_size)
predictions=tf.nn.softmax(predictions)[:,:,-1]

test_df[target]=np.squeeze(predictions.numpy())
print(time.time()-start)

In [ ]:
import riiideducation
env = riiideducation.make_env()
iter_test = env.iter_test()
prior_test_df = None

In [ ]:
%%time

for (test_df, sample_prediction_df) in iter_test: 
    if prior_test_df is not None:
        prior_test_df[target] = eval(test_df['prior_group_answers_correct'].iloc[0])
        prior_test_df = prior_test_df[prior_test_df[target] != -1].reset_index(drop=True)
        user_ids = prior_test_df['user_id'].values
        content_ids = prior_test_df['content_id'].values
        targets = prior_test_df[target].values 
        for user_id, answered_correctly,content_id in zip(user_ids,targets,content_ids):
            #when train ,content_id add 1
            if user_id in train_dataset.keys():
                train_dataset[user_id][0]=np.concatenate((train_dataset[user_id][0],[content_id+1]))
                train_dataset[user_id][1]=np.concatenate((train_dataset[user_id][1],[answered_correctly]))
            else:
                train_dataset[user_id]=[np.array([content_id+1]),np.array([answered_correctly])]
             
            
        
    prior_test_df = test_df.copy()
    
    test_df = test_df[test_df['content_type_id'] == 0].reset_index(drop=True)
    
    user_ids = test_df['user_id'].values
    content_ids = test_df['content_id'].values
    
   
 
    encoder_inputs, tar_inps=getpredatas(user_ids,content_ids)
    enc_padding_mask, combined_mask, dec_padding_mask = create_masks(
        encoder_inputs, tar_inps)
    predictions, attention_weights = transformer(encoder_inputs, 
                                                 tar_inps,
                                                 False,
                                                 enc_padding_mask,
                                                 combined_mask,
                                                 dec_padding_mask)
    predictions = predictions[: ,-1:, :]  # (batch_size, 1, vocab_size)
    predictions=tf.nn.softmax(predictions)[:,:,-1]
    
    test_df[target]=np.squeeze(predictions.numpy())
       
    #    
    env.predict(test_df[['row_id', target]])
    